## Парсинг HTML. BeautifulSoup

В данном ноутбуке реализован парсинг основной информации по вакансиям с сайтов hh.ru и superjob.ru.
Пользователь может передать через input ключевые слова для поиска и регион (поиск работает для 12-ти крупных российских городов). Приложение анализирует все доступные страницы поиска.
Результат работы парсера сохраняется в \*.csv файл, содержащий следующие поля:

- ID вакансии
- Должность
- Работодатель
- Зарплата (отдельно минимум, максимум и валюта)
- Cсылка на вакансию
- Источник вакансии
- Описание вакансии

In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
import json
import pandas as pd

In [2]:
# параметры поиска
query = input('Введите запрос: ').lower()
area = input('Введите город для поиска: ').lower()

Введите запрос: прораб
Введите город для поиска: москва


In [3]:
# словарь для результатов поиска
vacancies = {
    'query': f'{query}_{area}',
    'results': []
}

In [4]:
# словари для трансформации региона поиска
hh_area_dict = {
    'москва': '1',
    'санкт-петербург': '2',
    'екатеринбург': '3',
    'новосибирск': '4',
    'нижний новгород': '66',
    'казань': '88',
    'воронеж': '26',
    'волгоград': '24',
    'ростов-на-дону': '76',
    'краснодар': '53',
    'уфа': '99', 
    'хабаровск': '102'
}

sj_area_dict = {
    'москва': 'москва',
    'санкт-петербург': 'spb',
    'екатеринбург': 'ekaterinburg',
    'новосибирск': 'nsk',
    'нижний новгород': 'nn',
    'казань': 'kazan',
    'воронеж': 'voronezh',
    'волгоград': 'volgograd',
    'ростов-на-дону': 'rnd',
    'краснодар': 'krasnodar',
    'уфа': 'ufa', 
    'хабаровск': 'habarovsk'
}

In [5]:
# преобразовываем input в понятный парсеру
if area in ['москва', 'мск']:
    hh_area = hh_area_dict['москва']
    sj_area = sj_area_dict['москва']
    
elif area in ['санкт-петербург', 'спб']:
    hh_area = hh_area_dict['санкт-петербург']
    sj_area = sj_area_dict['санкт-петербург']
    
elif area in ['екатеринбург', 'екб']:
    hh_area = hh_area_dict['екатеринбург']
    sj_area = sj_area_dict['екатеринбург']
    
elif area == 'новосибирск':
    hh_area = hh_area_dict['новосибирск']
    sj_area = sj_area_dict['новосибирск']

elif area == 'нижний новгород':
    hh_area = hh_area_dict['нижний новгород']
    sj_area = sj_area_dict['нижний новгород']

elif area == 'казань':
    hh_area = hh_area_dict['казань']
    sj_area = sj_area_dict['казань']

elif area == 'воронеж':
    hh_area = hh_area_dict['воронеж']
    sj_area = sj_area_dict['воронеж']   

elif area == 'волгоград':
    hh_area = hh_area_dict['волгоград']
    sj_area = sj_area_dict['волгоград']

elif area in ['ростов-на-дону', 'рнд']:
    hh_area = hh_area_dict['ростов-на-дону']
    sj_area = sj_area_dict['ростов-на-дону']

elif area == 'краснодар':
    hh_area = hh_area_dict['краснодар']
    sj_area = sj_area_dict['краснодар']  

elif area == 'уфа':
    hh_area = hh_area_dict['уфа']
    sj_area = sj_area_dict['уфа']

elif area == 'хабаровск':
    hh_area = hh_area_dict['хабаровск']
    sj_area = sj_area_dict['хабаровск']  
    
else:
    print('Неизвестный регион')

In [6]:
hh = 'https://hh.ru/'
sj = 'https://www.superjob.ru'

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:74.0) Gecko/20100101 Firefox/74.0'}

# задаем счетчики страниц - у hh - c 0, у sj - c 1
hh_page_count = range(100)
sj_page_count = range(1,100)

# создаем паттерны для извлечения id вакансии
hh_id_pattern = re.compile(r'https://[^/]+/vacancy/(\d+)')
sj_id_pattern = re.compile(r'/vakansii/[\w+\-\w+]+\-(\d+)')

**hh.ru parsing**

In [7]:
for i in hh_page_count:
    hh_link = f'{hh}search/vacancy?area={hh_area}&st=searchVacancy&text={query}&page={i}'
    hh_page = requests.get(hh_link, headers=headers)
    
    if not hh_page.ok:
        break
    
    # формируем список элементов страницы, содержащих вакансии
    hh_soup = bs(hh_page.text,'lxml')
    vacancies_block = hh_soup.find_all('div', {'class': 'vacancy-serp'})[0]
    vacancies_list = vacancies_block.find_all('div', {'class': 'vacancy-serp-item'})
    
    # определяем условие выхода из цикла (вакансий больше нет)
    if len(vacancies_list) == 0:
        break
    
    # итерируемся по списку, содержащем вакансии
    for vacancy in vacancies_list:
        vacancy_data = {}

        
        # название вакансии
        title = vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0].getText()

        
        # работадатель
        employer_el = vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-employer'})
        
        # обрабатываем случай, когда работодатель указан описательно (нет ссылки на компанию)
        if employer_el:
            employer = employer_el[0].getText()
        else:
            employer = vacancy.findChildren('div', {'class': 'vacancy-serp-item__meta-info'})[0].getText()              

         
        #  зарплата
        min_salary, max_salary, currency = None, None, None
        
        salary_el = vacancy.findChildren('span', {'data-qa': "vacancy-serp__vacancy-compensation"})
        
        if salary_el:
            # получаем значения из строки
            split_range = re.split('-', salary_el[0].getText())
            if len(split_range) == 2:
                min_salary = ''.join(re.findall('\d+', split_range[0]))
                items = re.findall('\w+', split_range[1])
                max_salary = ''.join(items[:-1])
                currency = items[-1] 
            else:
                items = re.findall('\w+', split_range[0])
                if items[0] == 'от':
                    min_salary = ''.join(items[1:-1])
                else:
                    max_salary = ''.join(items[1:-1])          
                currency = items[-1]
               
        # ссылка
        vac_url = vacancy.findChildren('a', {'data-qa': 'vacancy-serp__vacancy-title'})[0]['href']

        # id вакансии
        vac_id = f'hh_{hh_id_pattern.findall(vac_url)[0]}'
              
        # описание
        desc1 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_responsibility'})[0].getText()
        desc2 = vacancy.findChildren('div', {'data-qa': 'vacancy-serp__vacancy_snippet_requirement'})[0].getText()
        
        
        # записываем полученные значения в словарь по каждой вакансии
        vacancy_data['id'] = vac_id
        vacancy_data['title'] = title
        vacancy_data['employer'] = employer
        vacancy_data['min_salary'] = min_salary
        vacancy_data['max_salary'] = max_salary
        vacancy_data['currency'] = currency
        vacancy_data['url'] = vac_url
        vacancy_data['source'] = hh
        vacancy_data['short_desc'] = desc1 + ' ' + desc2
        
        # пополняем основной словарь вакансий
        vacancies['results'].append(vacancy_data)
        

**Superjob.ru parsing**

In [8]:
for i in sj_page_count:
    
    # обрабатываем случай, когда регион 'москва'
    if sj_area == 'москва':
        sj_link = f'{sj}/vacancy/search/?keywords={query}&geo[t][0]=4&page={i}'
    else:
        sj_link = f'https://{sj_area}.superjob.ru/vacancy/search/?keywords={query}&page={i}'
    
    sj_page = requests.get(sj_link, headers=headers)

    if sj_page.ok:
        
        # записываем нужные классы в переменные для лаконичности
        vac_block = '_1Ttd8 _2CsQi'
        vac_class = '_3zucV f-test-vacancy-item _3j3cA RwN9e _3tNK- _1NStQ _1I1pc'        
        title_url_class = '_3mfro CuJz5 PlM3e _2JVkc _3LJqf'
        salary_class = '_3mfro _2Wp8I _31tpt f-test-text-company-item-salary PlM3e _2JVkc _2VHxz'
        employer_class = '_3mfro _3Fsn4 f-test-text-vacancy-item-company-name _9fXTd _2JVkc _2VHxz _15msI'
        location = '_3mfro f-test-text-company-item-location _9fXTd _2JVkc _2VHxz'
        desc_class = '_3mfro _3V-Qt _9fXTd _2JVkc _2VHxz'
        
        # формируем список элементов страницы, содержащих вакансии
        sj_soup = bs(sj_page.text,'lxml')
        vacancies_block = sj_soup.find_all('div', {'class': vac_block})[0]
        vacancies_list = vacancies_block.find_all('div', {'class': vac_class})
    
        # определяем условие выхода из цикла (вакансий больше нет)
        if len(vacancies_list) == 0:
            break
        
        # итерируемся по списку, содержащем вакансии
        for vacancy in vacancies_list:
            vacancy_data = {}
          
        
            # название вакансии
            title = vacancy.findChildren('div', {'class': title_url_class})
            
            # обрабатываем случай, когда title лежит не в span, а в h2
            if title:
                title = title[0].getText()
            else:
                title = vacancy.findChildren('h2', {'class': title_url_class})[0].getText()
            
            
            # работодатель
            employer_el = vacancy.findChildren('span', {'class':employer_class})
            
            # обрабатываем случай, когда работодатель не указан
            if employer_el:
                employer = employer_el[0].getText()
            else:
                employer = None
                
                
            # зарплата
            salary_el = vacancy.findChildren('span', {'class': salary_class})[0].getText()
            
            min_salary, max_salary, currency = None, None, None
            
            if salary_el != 'По договорённости':
                #разбиваем по тире, заменяем неразрывные пробелы
                split_range = re.split('—', salary_el.replace('\xa0', ' '))

                # диапазон
                if len(split_range) == 2:
                    min_salary = ''.join(re.findall('\d+', split_range[0]))
                    items = re.findall('\w+', split_range[1])
                    max_salary = ''.join(items[:-1])
                    currency = items[-1] 
                else:
                    # разбиваем по пробелам список с одним элементом
                    items_one = re.findall('\w+', split_range[0])
                    # от или до
                    if items_one[0] == 'от':
                        min_salary = ''.join(items_one[1:-1])
                    elif items_one[0] == 'до':
                        max_salary = ''.join(items_one[1:-1])  
                    # зп одним числом
                    else:
                        min_salary = ''.join(items_one[:-1]) 
                        max_salary = min_salary
                    currency = items_one[-1]

            # ссылка
            url = vacancy.findChildren('div', {'class': title_url_class})
            
            # обрабатываем случай, когда url лежит не в span, а в h2
            if url:
                url = url[0].find('a')['href']
                vac_url = sj + url             
            else:
                url = vacancy.findChildren('h2', {'class': title_url_class})[0].find('a')['href']
                vac_url = sj + url
                print(url)

            # id вакансии
            vac_id = f'sj_{sj_id_pattern.findall(url)[0]}'     
        
            # описание
            desc_el = vacancy.findChildren('span', {'class': desc_class})
            
            # обрабатываем случай, когда описания вакансии нет
            if desc_el:
                desc = desc_el[0].getText()
            else:
                desc = None
             
            
            # записываем полученные значения в словарь по каждой вакансии
            vacancy_data['id'] = vac_id
            vacancy_data['title'] = title
            vacancy_data['employer'] = employer
            vacancy_data['min_salary'] = min_salary
            vacancy_data['max_salary'] = max_salary
            vacancy_data['currency'] = currency
            vacancy_data['url'] = vac_url
            vacancy_data['source'] = sj
            vacancy_data['short_desc'] = desc
            
            # пополняем основной словарь вакансий
            vacancies['results'].append(vacancy_data)

In [9]:
# обрабатываем случай, если по запросу ничего не найдено
hh_counter = 0
sj_counter = 0

for i in vacancies['results']:
    if i['source'] == hh:
        hh_counter += 1
    else:
        sj_counter +=1
        
if hh_counter == 0 and sj_counter == 0:
    print('По данному запросу ничего не найдено')

In [10]:
hh_counter, sj_counter

(594, 68)

In [11]:
print(len(vacancies['results']))

662


In [12]:
vacancies_df = pd.DataFrame(vacancies['results'])
vacancies_df.head()

,id,title,employer,min_salary,max_salary,currency,url,source,short_desc
0,hh_36571024,Производитель работ (прораб),ООО Стройжилпром,80000,100000,руб,https://kolomna.hh.ru/vacancy/36571024?query=%...,https://hh.ru/,Грамотное распределение трудовых ресурсов перс...
1,hh_36627671,Прораб отделочных работ,ООО РУССТРОЙ,80000,None,руб,https://kolomna.hh.ru/vacancy/36627671?query=%...,https://hh.ru/,Работа в компании ген подрядчика. Все объекты ...
2,hh_36566630,Прораб электромонтажных работ,ООО Кит Энергетика,65000,80000,руб,https://kolomna.hh.ru/vacancy/36566630?query=%...,https://hh.ru/,Производство работ по электромонтажу осветител...
3,hh_35012595,Прораб,ООО Стройком,None,None,None,https://kolomna.hh.ru/vacancy/35012595?query=%...,https://hh.ru/,Ведение отделочных работ. Контроль за ходом ст...
4,hh_36347654,Прораб/Прораб отделочных работ,ООО Мастера Ремонта,100000,200000,руб,https://kolomna.hh.ru/vacancy/36347654?query=%...,https://hh.ru/,"СВОЙ ОТДЕЛ ЗАКУПОК,СВОЙ СКЛАД ЧЕРНОВЫХ МАТЕРИА..."


In [13]:
vacancies_df.to_csv(f'{vacancies["query"]}.csv', index=False)